<a href="https://colab.research.google.com/github/geeky-programer/LLM/blob/main/Intro_langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from langchain.llms import OpenAI

In [ ]:
from dotenv import load_dotenv
load_dotenv()
import os

In [ ]:
llm=OpenAI(temperature=0.6)

In [ ]:
text = "what is the capital of India"

print(llm.predict(text))

In [ ]:
from langchain import HuggingFaceHub
llm_hug = HuggingFaceHub(repo_id="google/flan-t5-large",model_kwargs={"temperature":0,"max_length":64})

In [ ]:
llm_hug.predict(text)

In [ ]:
## prompt templates

from langchain.prompts import PromptTemplate

prompt_template = PromptTemplate(input_variables=['country'], template="Tell me the capital of this {country}")

prompt_template.format(country='India')

from langchain.chains import LLMChain
chain = LLMChain(llm=llm, prompt = prompt_template)
chain.run("India")

## chain --> LLM model + prompt template


In [ ]:
### multiple chains & simple sequential chains


capital_template = PromptTemplate(input_variables=["country"], template = "Please tell me the capital of {country}")

capital_chain = LLMChain(llm=llm, prompt=capital_template)


famous_template = PromptTemplate(input_variables=["capital"], template = "suggest me places to visit {capital}")

famous_chain = LLMChain(llm=llm,prompt=famous_template)




In [ ]:
from langchain.chains import SimpleSequentialChain
chain = SimpleSequentialChain(chains =[capital_chain, famous_chain])
chain.run("India")


In [ ]:
capital_template = PromptTemplate(input_variables=["country"], template = "Please tell me the capital of {country}")

capital_chain = LLMChain(llm=llm, prompt=capital_template, output_key="capital")

famous_template = PromptTemplate(input_variables=["capital"], template = "suggest me places to visit {capital}")

famous_chain = LLMChain(llm=llm,prompt=famous_template,output_key="places")


In [ ]:
from langchain.chains.sequential import SequentialChain
chain = SequentialChain(chains = [capital_chain, famous_chain], input_variables=['country'], output_variables = ['capital', 'places'])

In [ ]:
chain({'country':'India'})

In [ ]:
###chat bots with Openai

In [ ]:
from langchain.chat_models import ChatOpenAI

In [ ]:
from langchain.schema import HumanMessage,SystemMessage,AIMessage

In [ ]:
chatllm=ChatOpenAI(temperature=0.6,model='gpt-3.5-turbo')

In [ ]:
chatllm([
SystemMessage(content="Yor are a professor AI learning"),
HumanMessage(content="Please provide insights on deep learning and a guide to learn it")
])

In [ ]:
## prompt template ++ LLM ++ Output parser

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate
from langchain.schema import BaseOutputParser

In [ ]:
class Commaseparatedoutput(BaseOutputParser):
    def parse(self, text:str):
      return text.strip().split(',')

In [ ]:
template = "you are a helpful assistant. When the user gives any input you should generated 5 words synonyms which are comma separated"
human_template = "{text}"
chatprompt = ChatPromptTemplate.from_messages([('system',template),('human',human_template)])

In [ ]:
chain = chatprompt|chatllm|Commaseparatedoutput()

In [ ]:
chain.invoke({'text':'intelligent'})